<a href="https://colab.research.google.com/github/eacb93-coder/adm/blob/main/triagem_emails_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
lista_emails = []

lista_emails.append("Prezados, o vencimento do boleto da TechSolutions foi ontem e não recebemos o comprovante. Podem verificar?")

lista_emails.append("Bom dia. Gostaria de saber até que dia preciso enviar os atestados médicos para fechamento do ponto deste mês.")

lista_emails.append("Urgente: O estoque de papel A4 acabou e a impressora do 3º andar está parada. Precisamos de reposição imediata.")

lista_emails.append("Olá, gostaria de agendar uma reunião para apresentar nossa nova proposta de consultoria. Tenho disponibilidade na terça.")

lista_emails.append("A apresentação de resultados trimestrais ficou excelente! A diretoria elogiou muito a organização dos dados.")

print("--- Caixa de Entrada (Administrativa) Criada ---")
for email in lista_emails:
    print(f"📁 {email}")

--- Caixa de Entrada (Administrativa) Criada ---
📁 Prezados, o vencimento do boleto da TechSolutions foi ontem e não recebemos o comprovante. Podem verificar?
📁 Bom dia. Gostaria de saber até que dia preciso enviar os atestados médicos para fechamento do ponto deste mês.
📁 Urgente: O estoque de papel A4 acabou e a impressora do 3º andar está parada. Precisamos de reposição imediata.
📁 Olá, gostaria de agendar uma reunião para apresentar nossa nova proposta de consultoria. Tenho disponibilidade na terça.
📁 A apresentação de resultados trimestrais ficou excelente! A diretoria elogiou muito a organização dos dados.


In [2]:
print("--- Salvando e-mails no arquivo 'emails_recebidos.txt'... ---")

with open("emails_recebidos.txt", "w", encoding="utf-8") as arquivo:
    for email in lista_emails:
        arquivo.write(email + "\n")

print("✅ Arquivo criado com sucesso! Verifique na pastinha 📁 ao lado.")

--- Salvando e-mails no arquivo 'emails_recebidos.txt'... ---
✅ Arquivo criado com sucesso! Verifique na pastinha 📁 ao lado.


In [3]:
lista_emails_limpos = []

print("--- Lendo os novos e-mails administrativos... ---")

with open("emails_recebidos.txt", "r", encoding="utf-8") as arquivo:
    for linha in arquivo:
        lista_emails_limpos.append(linha.strip())

print(f"✅ {len(lista_emails_limpos)} e-mails carregados para análise.")

--- Lendo os novos e-mails administrativos... ---
✅ 5 e-mails carregados para análise.


In [4]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('alura')
if not api_key:
    api_key = userdata.get('GEMINI_API_KEY')

if not api_key:
    print("❌ ERRO: A chave de API não foi encontrada. Verifique o painel Secrets.")
else:
    genai.configure(api_key=api_key)

    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        print("✅ Autenticação e Objeto 'model' criados.")
    except Exception as e:
        print(f"❌ ERRO ao carregar o modelo: {e}")

✅ Autenticação e Objeto 'model' criados.


In [5]:
import time

print("--- Iniciando Triagem de E-mails Corporativos... ---")

dados_para_relatorio = []

for email_original in lista_emails_limpos:
    try:
        prompt_admin = f"""
        Você é um assistente administrativo eficiente. Analise a demanda abaixo:
        '{email_original}'

        Classifique em:
        1. DEPARTAMENTO: (Financeiro, RH, Compras, Comercial, Diretoria)
        2. PRIORIDADE: (ALTA, MEDIA, BAIXA)

        Regras de Prioridade:
        - Boletos vencidos, falta de insumos ou paradas operacionais = ALTA
        - Prazos futuros ou agendamentos = MEDIA
        - Elogios ou informativos = BAIXA

        Responda ESTRITAMENTE neste formato: DEPARTAMENTO | PRIORIDADE
        """

        response = model.generate_content(prompt_admin)

        texto_resposta = response.text.strip()
        partes = texto_resposta.split('|')

        if len(partes) >= 2:
            departamento = partes[0].strip()
            prioridade = partes[1].strip()

            dados_para_relatorio.append({
                "EMAIL_ORIGINAL": email_original,
                "DEPARTAMENTO": departamento,
                "PRIORIDADE": prioridade
            })

            print(f"   🏢 Encaminhado para: {departamento:15} | 🚨 {prioridade}")
        else:
            print(f"⚠️ Resposta da IA fora do padrão: {texto_resposta}")

        time.sleep(1)

    except Exception as e:
        print(f"❌ Erro no email: {email_original[:20]}... Detalhe: {e}")

print("\n✅ Triagem concluída! Os dados estão prontos para o Excel (CSV).")

--- Iniciando Triagem de E-mails Corporativos... ---
   🏢 Encaminhado para: Financeiro      | 🚨 ALTA
   🏢 Encaminhado para: RH              | 🚨 MEDIA
   🏢 Encaminhado para: Compras         | 🚨 ALTA
   🏢 Encaminhado para: COMERCIAL       | 🚨 MEDIA
   🏢 Encaminhado para: Financeiro      | 🚨 BAIXA

✅ Triagem concluída! Os dados estão prontos para o Excel (CSV).


In [6]:
import pandas as pd
from IPython.display import display

print("--- Gerando Relatório Gerencial... ---")

df_relatorio = pd.DataFrame(dados_para_relatorio)

df_relatorio.index = df_relatorio.index + 1

df_relatorio.to_csv("Relatorio_Triagem_Adm.csv", index=False, encoding="utf-8")

display(df_relatorio)

print("\n✅ Sucesso! O arquivo 'Relatorio_Triagem_Adm.csv' está salvo na pasta 📁.")

--- Gerando Relatório Gerencial... ---


,EMAIL_ORIGINAL,DEPARTAMENTO,PRIORIDADE
1,"Prezados, o vencimento do boleto da TechSoluti...",Financeiro,ALTA
2,Bom dia. Gostaria de saber até que dia preciso...,RH,MEDIA
3,Urgente: O estoque de papel A4 acabou e a impr...,Compras,ALTA
4,"Olá, gostaria de agendar uma reunião para apre...",COMERCIAL,MEDIA
5,A apresentação de resultados trimestrais ficou...,Financeiro,BAIXA



✅ Sucesso! O arquivo 'Relatorio_Triagem_Adm.csv' está salvo na pasta 📁.
